# A Closer Look at Python List Comprehensions

Inspired by Trey Hunner's blog post (https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/), I'm taking a bit deeper look into list comprehensions.

We start with a typical task in Python - transform an existing list into a new one.

In [11]:
def func(item):
    return item > 0

def f1():
    old_list = [0, 1, 2, -1, -3, 2]
    new_list = []
    for item in old_list:
        if func(item):
            new_list.append(item)
    return new_list

Let's use the dis module to examine what's going on at the byte code level.

In [14]:
import dis
dis.dis(f1)

  5           0 LOAD_CONST               1 (0)
              3 LOAD_CONST               2 (1)
              6 LOAD_CONST               3 (2)
              9 LOAD_CONST               5 (-1)
             12 LOAD_CONST               6 (-3)
             15 LOAD_CONST               3 (2)
             18 BUILD_LIST               6
             21 STORE_FAST               0 (old_list)

  6          24 BUILD_LIST               0
             27 STORE_FAST               1 (new_list)

  7          30 SETUP_LOOP              39 (to 72)
             33 LOAD_FAST                0 (old_list)
             36 GET_ITER
        >>   37 FOR_ITER                31 (to 71)
             40 STORE_FAST               2 (item)

  8          43 LOAD_GLOBAL              0 (func)
             46 LOAD_FAST                2 (item)
             49 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             52 POP_JUMP_IF_FALSE       37

  9          55 LOAD_FAST                1 (new_list)
             58 

Whew! That's a lot of byte codes. The first column is the line number from the code fragment we entered. There's not much documentation on the rest of the output, but it's simple enough to see what's going on.

First, LOAD_CONST loads a constant into the stack. BUILD_LIST takes the constants and builds a list, which STORE_FAST pushes on the stack. This is the byte code corresponding to line 5. We next begin with line 6, which creates an empty list.

Line 7 sets up the loop, which ends at bytecode line 72 in our listing, and LOAD_FAST pushes a reference onto the stack. Next, get the iterator, and we start the loop. FOR_ITER calls next() on the iteratable object (in this case a list), and pushes it onto the stack.

There are a few other bookkeeping calls here, but note the LOAD_ATTR byte code. This loads the append() function from the virtual table for the list object. As we shall see below, this is the biggest bottleneck for performance.

The complete list of byte codes for version 2.7 is https://docs.python.org/2/library/dis.html, in case you are interested.